# Script for creating synthetic, 3D mixture-of-Gaussians data

In [1]:
%matplotlib inline

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import h5py
from scipy.stats import multivariate_normal
from scipy.io import savemat
from IPython.display import HTML

### Parameters

In [3]:
# File to write to
OUT_FPATH_H5 = '/home/mn2822/Desktop/WormTracking/data/synthetic/gmm_data_3d.h5'
OUT_FPATH_MAT = '/home/mn2822/Desktop/WormTracking/data/synthetic/gmm_data_3d.mat'

# Image size 
IMG_SIZE = [100, 50, 20]

# Image size limits
IMG_XLIM = [0, 100]
IMG_YLIM = [0, 50]
IMG_ZLIM = [0, 20]

# Number of samples
T = 50

# Sample rate (Hz)
SMP_RATE = 10

# Number of mixture components
K = 10

# Number of 'cycles' spanning worm (does not need to be integer)
N_CYCLES = 0.75

# Frequency of worm movement (Hz)
FREQ = 0.5

# Amplitude of worm movement (image units)
AMP = 12.5

# Scale of isotropic covariance matrix for GMM
COV_SCL = 5.0

# Flag for whether or not to add noise
ADD_NOISE = False

# Noise level (stddev of Gaussian noise)
NOISE_STD = 1e-4

### Create time series of mean positions

In [4]:
# X-values of means are equally spaced; don't change in time
means_x = np.linspace(IMG_XLIM[0], IMG_XLIM[1], K + 2);
means_x = means_x[1:K+1];
means_x = np.tile(means_x, [T, 1]);

# Y-values of means oscillate in time
phases = np.linspace(0, N_CYCLES * 2 * np.pi, K)
phases = phases[:, np.newaxis]
offset = IMG_YLIM[0] + (IMG_YLIM[1] - IMG_YLIM[0]) / 2;
rads = (2 * np.pi * FREQ / SMP_RATE) * np.arange(0, T);
rads = rads[:, np.newaxis]
means_y = offset + AMP * np.sin(rads + phases.T);

# Z-values of means are same for all components and time points
means_z = np.ones((T, K)) * (IMG_ZLIM[0] + IMG_ZLIM[1]) / 2

### Use mean positions to create time series of GMM densities

In [5]:
def img_pdf(x, mu, cov):
    """Compute GMM PDF for given means and variance value."""
    
    n_comp = mu.shape[0]
    px_nn = np.zeros((x.shape[0], x.shape[1], x.shape[2]))
    
    for k in range(n_comp):
        px_nn += multivariate_normal.pdf(x, mu[k, :], cov)
        
    return px_nn / np.sum(px_nn)

In [6]:
# Covariance matrix is isotropic, with scale determined by parameter
sigma = COV_SCL * np.eye(3);

# Create grid for evaluating densities on
xg, yg, zg = np.mgrid[-IMG_XLIM[0]:IMG_XLIM[1], -IMG_YLIM[0]:IMG_YLIM[1], -IMG_ZLIM[0]:IMG_ZLIM[1]]
grid = np.stack((xg, yg, zg), axis=-1)

# Evaluate densities to get sequence of images
data = np.zeros((IMG_SIZE[0], IMG_SIZE[1], IMG_SIZE[2], T));
for t in range(T):
    
    # Collect means for all components at time t into array
    mu = np.vstack((means_x[t, :], means_y[t, :], means_z[t, :])).T
    
    # Compute GMM PDF values at grid points
    px = img_pdf(grid, mu, sigma)
    
    # Reshape PDF vector into 3D image
    data[:, :, :, t] = px

### Play synthetic data as video

In [7]:
# Create list of image plots
fig = plt.figure()
ims = []
for t in range(T):
    frame_mp = np.max(data[:, :, :, t], 2)
    im = plt.imshow(frame_mp.T, animated=True)
    ims.append([im])
    
# Compile images into animation object
ani = animation.ArtistAnimation(
    fig, ims, interval=150, blit=True, repeat_delay=1000)

# Prevent double-display of animation
plt.close()

# Display animation in notebook
HTML(ani.to_html5_video())

### Save data to H5 file

In [9]:
means = np.stack((means_x, means_y, means_z), axis=-1)
weights = np.ones((T, K)) * (1 / K)

with h5py.File(OUT_FPATH_H5, 'w') as f:
    
    f.create_dataset('video', data=data)
    f.create_dataset('means', data=means)
    f.create_dataset('weights', data=weights)
    f.create_dataset('cov', data=sigma)
    
    f.attrs['source'] = 'create_gmm_data_3d.ipynb'

### Save data to MAT file

In [11]:
mat_dict = {
    'video': data,
    'means': np.array(means),
    'weights': np.array(weights),
    'cov': sigma
}
savemat(OUT_FPATH_MAT, mat_dict)